In [1]:
#Imports
import resources
import pandas as pd
from blcalc.soilproperty import SoilProperty
from blcalc.material import LayerSoil
from blcalc.footing import Footing, FootingType, FootingData
from blcalc.assembly import Assembly
from blcalc.solver import Solver, Methods

In [2]:
filepath = resources.secrets_dir() / "calc-merged.xlsx"

In [3]:
df = pd.read_excel(str(filepath))
df.columns

Index(['Depth', 'R.L of GWT', 'SPT(N)', 'N', 'Soil Type', 'Qu', 'Cu', 'phi',
       'Cb', 'Cs', 'Eh', 'Cr', 'gamma', 'w', 'G', 'Dry unit wt',
       'Saturated Unit weight', 'Vertical Effective Strees',
       'Total Effective Stress', 'CN', 'N60', 'FC', 'del(N1)60', '(N1)60CS',
       'CRR', 'Shear Reduction Factor', 'MSF', 'fake-Csigma', 'C-sigma',
       'fake-K-sigma', 'K-sigma', 'a/g', 'CSR', 'FS', 'Fz', 'Wz', 'Fz*wz*d',
       'lpi'],
      dtype='object')

In [4]:
fields_list=[] # (from,to)
starting = None
for i in range(len(df.index)):
    c=df['N60'][i]
    if(not pd.isnull(c) and starting is None):
        starting = i
    if (not starting is None) and pd.isnull(c):
        fields_list.append([starting, i])
        starting = None
fields_list.append([starting, len(df.index)])
print(len(fields_list), ' total borehole logs.')

104  total borehole logs.


In [5]:
locations_log = []
for idx,fl in enumerate(fields_list):
    i=fl[0]-2
    if not (pd.isnull(df['Depth'][i]) and pd.isnull(df['R.L of GWT'][i])):
        indx=[(df['Depth'][i],float(df['N'][i]),float(df['Soil Type'][i]))]
        locations_log.append(indx)
    locations_log[len(locations_log)-1].append(idx)
    fl.append(len(locations_log)-1)
print(len(locations_log), 'locations')

31 locations


In [6]:
for i in locations_log:
    print(i[0])

('Rastriya Banijya Bank: Thapathali, Kathmandu', 27.655862, 85.3064456)
('Building Complex: Anamnagar, Kathmandu', 27.6972818, 85.3274572)
('Building Complex: Bakhundol Lalitpur', 27.6830179, 85.3103347)
('Hindu vidhypeth: Balkumari, Lalitpur', 27.6711385, 85.3379345)
('DI Skin Health and Referral Center (P). Ltd.: Bansbari, Kathmandu', 27.7370376, 85.3335435)
('Building: Bhaisipati, Lalitpur', 27.648046, 85.2959632)
('Brihaspati Vidyasadan School: Gahanapokhari, Kathmandu', 27.7142534, 85.3295707)
('Green Hill City (P). Ltd. : Mulpani, Kathamndu ', 27.7157651, 85.386468)
('Building Site: Itachhe tol, Bhaktapur', 27.6728428, 85.4226077)
('Buddha Air (P). Ltd. : Jawalakhel, Lalitpur ', 27.6735721, 85.3118742)
('Tamrakar Samaj: Jawalakhel', 27.6747489, 85.312196)
('Tangal, Kathmandu', 27.7200932, 85.3262587)
('Harihar Bhavan, Lalitpur', 27.6804712, 85.3111097)
('Janamaitri Campus: Kuleswore, Kathmandu', 27.6917301, 85.290048)
('Amrit Science Campus: Lainchour, Kathmandu', 27.717759, 85.3

In [7]:
#get material info from table

def get_layer(row):
    layer = {}
    layer[SoilProperty.SPT_N] = df['SPT(N)'][row]
    layer[SoilProperty.depth] = df['Depth'][row]
    layer[SoilProperty.gamma] = df['gamma'][row]
    layer[SoilProperty.GI] = df['Soil Type'][row]
    if not pd.isnull(df['phi'][row]):
        layer[SoilProperty.phi] = df['phi'][row]
    if not pd.isnull(df['Qu'][row]):
        layer[SoilProperty.qu] = df['Qu'][row]
    if not pd.isnull(df['Cu'][row]):
        layer[SoilProperty.cu] = df['Cu'][row]
    layer[SoilProperty.water_per] = df['w'][row]
    layer[SoilProperty.G] = df['G'][row]
    layer[SoilProperty.FC] = df['FC'][row]
    return layer

def get_material(idx, isliquifaction=False):
    data=[]
    for i in range(fields_list[idx][0],fields_list[idx][1]):
        data.append(get_layer(i))
    water_depth = 0 
    if isliquifaction:
        water_depth = df['R.L of GWT'][fields_list[idx][0]]
    return LayerSoil(data, water_depth, idx)

In [8]:
#Get footing info
def get_footing(depth=1.5):
    footing = Footing()
    footing[FootingData.Type] = FootingType.Square
    footing[FootingData.Depth] = depth
    footing[FootingData.Width] = 2
    footing[FootingData.Length] = 2
    return footing

In [9]:
import csv

methods = []
for i in Methods:
    methods.append(i.value)

with open('../secrets/results.csv', 'w', newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(methods)
    for  ixx in range(len(fields_list)):
        #print(ixx, '/', len(fields_list))
        footing = get_footing()
        lay = get_material(ixx)
        lay2 = get_material(ixx,True)
        assembly = Assembly(footing, lay2)
        solver = Solver(assembly)
        lpi = solver.run([Methods.Liquifaction])[Methods.Liquifaction]
        for depth in [1.5,3,4.5]:
            footing = get_footing(depth)
            assembly = Assembly(footing, lay)
            solver = Solver(assembly)
            result = solver.run()
            result[Methods.Liquifaction] = lpi
            row = []
            for i in Methods:
                row.append(result[i])
            writer.writerow(row)
        writer.writerow([])